In [2]:
import pandas as pd
import requests
import json
import csv
import os
import time

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [77]:
def get_educ_school_acs_data(base_url, vars_str, fips_df, op_path, op_file, year, key, geo_type):
    """
    :param base_url: Different base url for 2000 and 2010 years
    :param fips_df: df with state and county fips codes
    :param op_file: op file name to which the data needs to be written
    :param census_year: year for which decennial census data
    :return:
    """
    
    # take a count variable to help distinguish 1st request from subsequent requests so that we take the variable
    # names only from the 1st call as the header names and skip varibale names from the subsequent calls.
    response = ''
    count = 0
    # open a new file with the required output file name. To overwrite a file and write some new data into the file, we can open the file in the w mode, which will delete the old data from the file.
    with open(f'{op_path}/{op_file}', 'a') as file_wrtr:
        # take a csv writer object to write to this output file
        op_file_writer = csv.writer(file_wrtr)
    
        state_fips = fips_df['state'].unique().astype(str)
        # Iterate over each row in the state_county fips df
        for st_fips in state_fips:
            ## create the request url for each state and fips code to get the data for all county subdivisions under it ##
            try:
                """ Use timer if needed """
#                 if count in [20, 40, 60, 80, 100]:
#                     time.sleep(2) # milliseconds

                if st_fips.__len__() < 2:
                    st_fips = '0' + st_fips
                
                print('st_fips: ', st_fips)
                    
                # cnty_fips = str(getattr(row, "county"))

                # https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=county%20subdivision:*&in=state:36&key=YOUR_KEY_GOES_HERE
                # url = f'{base_url}/{year}/acs/acs1?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:{cnty_fips}&key=aefa63e9ca460169068f42b9ebc3101db025882f'

                if geo_type == 'c_subd':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:*&key={key}'
                   # url = f'{base_url}/{year}/acs5?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:*'
                elif geo_type == 'place':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=place:*&in=state:{st_fips}&key={key}'
                elif geo_type='county':
                    url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=county:*&in=state:{st_fips}&key={key}'

                print(f'url at count{count}: ', url)

                headers = requests.utils.default_headers()
                headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36'
                response = requests.get(url, headers=headers)
                print('response: ', response)
                # load the json in to python object which would be list of all entries in the json object
                resp = json.loads(response.content)
                # If this is the first call(count=0), then write the entire content so that the 1st row with the variable names is the header
                if count == 0:
                    # iterate over respone python object and write each row to the csv
                    for res in resp:
                        op_file_writer.writerow(res)
                else:
                    # skipping 1st row which are the variable names starting from 2nd calls.
                    for res in resp[1:]:
                        op_file_writer.writerow(res)

                count += 1
                time.sleep(10)
                op_file_writer.writerow([])
                print('Total API Calls: ', count)
            except Exception as ex:
                print('Error response', response)
                print('Exception: ', ex)
                break



# final_df = create_final_twnshp_file(
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/twnshps_api',
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/new_census_townships_90_initial.csv')
#
# # sort the final df by state, county and then county subdivision to make sure they are in required ascending order. Default is ascending
# final_df_sorted = final_df.sort_values(by=['state', 'county', 'county subdivision'])
#
# # write the final df to a csv
# final_df_sorted.to_csv(
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/new_census_townships_90.csv',
#     index=False)

In [75]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/st_cnty_fips_2010.csv')

# get county subdivision level data
for year in years:
    print('year: ', year)
    get_educ_school_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E', 
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='county',
                             year = year,
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs/cnty',
                             op_file=f'educ_school_acs_initial_{year}.csv')

year:  2013
st_fips:  01
url at count0:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:01&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B1

response:  <Response [200]>
Total API Calls:  13
st_fips:  17
url at count13:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:17&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  14
st_fips:  18
url at count14:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

response:  <Response [200]>
Total API Calls:  26
st_fips:  30
url at count26:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:30&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  27
st_fips:  31
url at count27:  https://api.census.gov/data/2013/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

response:  <Response [200]>
Total API Calls:  1
st_fips:  02
url at count1:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:02&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  2
st_fips:  04
url at count2:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B1

response:  <Response [200]>
Total API Calls:  14
st_fips:  18
url at count14:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:18&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  15
st_fips:  19
url at count15:  https://api.census.gov/data/2017/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003

### Timing out after random calls - so trying to see if we can get data for 1 year from the last successful st_fips

In [103]:
st_cnty_fips_10 = pd.read_csv('/Users/salma/Research/dissertation_work/data/educ_schl_acs/st_fips_10_test.csv')

get_educ_school_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E',
                             fips_df=st_cnty_fips_10,
                             key='99d3a2679609fec77eb6c7de82884928a52310d4',
                             geo_type='county',
                             year = 2020,
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs/cnty',
                             op_file=f'educ_school_acs_initial_2020.csv')


st_fips:  45
url at count0:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E&for=county:*&in=state:45&key=99d3a2679609fec77eb6c7de82884928a52310d4
response:  <Response [200]>
Total API Calls:  1
st_fips:  46
url at count1:  https://api.census.gov/data/2020/acs/acs5?get=NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B1

### Get place level data

In [ ]:
for year in years:
    print('year: ', year)
    get_educ_school_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='NAME,B14001_002E,B14001_006E,B14001_007E,B14001_008E,B14001_009E,B14002_003E,B14002_013E,B14002_016E,B14002_019E,B14002_022E,B14002_027E,B14002_037E,B14002_040E,B14002_043E,B14002_046E,B15001_001E,B15001_002E,B15001_003E,B15001_011E,B15001_043E,B15001_044E,B15001_052E,B15002_001E,B15002_002E,B15002_019E,C15002A_001E,C15002A_002E,C15002A_007E,C15002B_001E,C15002B_002E,C15002B_007E,C15002H_001E,C15002H_002E,C15002H_007E,C15002I_001E,C15002I_002E,C15002I_007E', 
                             fips_df=st_cnty_fips_00,
                             key='aefa63e9ca460169068f42b9ebc3101db025882f',
                             geotype='place',
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs',
                             op_file=f'educ_school_acs_initial_{year}.csv')